In [1]:
## Tested using the following loaded modules
## module use /g/data/access/ngm/modules
## module load analysis3/21.10
import os, sys
workdir = "/g/data/xv83/users/bxn599/ACS/evaluation/"
os.chdir(workdir)
sys.path.append("./lib")
import glob
import xarray as xr
import numpy as np
import geopandas as gp
import datetime as dt
import lib_standards
import lib_spatial
import spatial_selection
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
importlib.reload(lib_standards)
import warnings
warnings.filterwarnings('ignore')
import csv
import barpa_drs_interface
import re

font = {'size'   : 12}
mpl.rc('font', **font)

In [2]:
GCMS=['EC-Earth-Consortium-EC-Earth3', 'NCC-NorESM2-MM']
SCENARIOS=['ssp370']
PERIOD='ACS_HISTORICAL'
LATEPERIODS = {'ACS_2020': '2020-2039', 'ACS_2040': '2040-2069', 'ACS_2060': '2060-2079', 'ACS_2080': '2080-2099'}
freq='mon'
VARS=['pr','tas']
REGION=['Australia', 'Queensland']
SEASONS=['all']

In [3]:
count = 0
header = [' ', 'Season', 'Variable', 'Project', 'Domain', 'Downscaler', 'Region', 'Model', 'Experiment', 'Variant', 'Hist-Epoch', 'Proj-Epoch', 'Change-Type', 'Value']
f = open('/g/data/xv83/users/bxn599/ACS/evaluation/test_raw_barpa_compare.csv','w')
writer = csv.writer(f)
writer.writerow(header)

117

In [4]:
for var in VARS:
    for region in REGION:
        for SEASON in SEASONS:
            if SEASON == 'all':
                season = 'annual'
            else:
                season = SEASON
            for gcm in GCMS:
                model = gcm
                experiment = barpa_drs_interface._GCM_ENS[gcm]
                if var == 'tas' and gcm != 'NCC-NorESM2-MM':
                    varh = 'tasmean'
                else:
                    varh = var
                dhist = barpa_drs_interface.get_barpa(gcm, 'historical', 'mon', varh, [dt.datetime(1995,1,1), dt.datetime(2014,12,31,23)], as_iris=False, lonrange=[112.0,156.2], latrange=[-44.5,-10.0],verbose=False)
                dhist = lib_standards.standardise_data(dhist, region='Australia', period=PERIOD, season=SEASON, compute=True)
                d0 = dhist[varh].mean(dim='time')
                d0 = lib_spatial.apply_region_mask(d0, region)
                for scenario in SCENARIOS:
                    dfut = barpa_drs_interface.get_barpa(gcm, scenario, 'mon', var, [dt.datetime(2015,1,1), dt.datetime(2099,12,31,23)], as_iris=False, lonrange=[112.0,156.2], latrange=[-44.5,-10.0],verbose=False)
                    for LATEPERIOD in LATEPERIODS:
                        futperiod=LATEPERIODS[LATEPERIOD]
                        dfuts = lib_standards.standardise_data(dfut, region='Australia', period=LATEPERIOD, season=SEASON, compute=True)
                        
                        d1 = dfuts[var].mean(dim='time')
                        d1 = lib_spatial.apply_region_mask(d1, region)
                        if var == 'pr':
                            d2 = (d1-d0)/d0*100 # percentage change
                        else:
                            d2 = (d1-d0)
                        
                        weights = np.cos(np.deg2rad(d2.lat))
                        weights.name = "weights"
                        area_avg = d2.weighted(weights)
                        area_avg = d2.mean(("lon", "lat"))
                        print(area_avg.values)
                        if var == 'pr':
                            data = [count, season, var, 'BARPA-R', 'AUS-15', 'n/a', region, model, scenario, experiment, '1995-2014', futperiod, 'percent', area_avg.values]
                        else:
                            data = [count, season, var, 'BARPA-R', 'AUS-15', 'n/a', region, model, scenario, experiment, '1995-2014', futperiod, 'absolute', area_avg.values]
                        writer.writerow(data)
                        count+=1

23.024948
25.50223
31.636969
67.219986
3.2697275
-5.588011
4.9893975
-2.3794913
17.388401
32.891953
42.381653
52.357708
0.15894692
-2.464945
1.4863088
-1.3288722
0.8166627
1.4585027
2.5884793
3.622257
0.29525998
1.2303035
1.8812423
2.816336
0.9220908
1.5384296
2.774844
3.8323882
0.57881933
1.4128717
2.0607326
3.1262026


In [5]:
f.close()